In [ ]:
import boto3
import pendulum

In [ ]:
s3 = boto3.resource('s3')
private_bucket = s3.Bucket('chn-ghost-buses-private')
public_bucket = s3.Bucket('chn-ghost-buses-public')

In [ ]:
# copy raw data from the private bucket to the public bucket 
start_date = '2022-08-10'
end_date = '2022-10-08'

date_range = [d for d in pendulum.period(pendulum.from_format(start_date, 'YYYY-MM-DD'), pendulum.from_format(end_date, 'YYYY-MM-DD')).range('days')]

In [ ]:
# copy fully raw JSON data from private to public
for day in date_range:
    date_str = day.to_date_string()
    print(f"Processing {date_str} at {pendulum.now().to_datetime_string()}")
    objects = private_bucket.objects.filter(Prefix = f'bus_data/{date_str}')
    for obj in objects:
        public_bucket.copy({'Bucket': private_bucket.name, 'Key': obj.key}, obj.key)

In [ ]:
# copy daily CSVs from private to public
for day in date_range:
    date_str = day.to_date_string()
    print(f"Processing {date_str} at {pendulum.now().to_datetime_string()}")
    data_objects = private_bucket.objects.filter(Prefix = f'bus_full_day_data_v2/{date_str}.csv')
    error_objects = private_bucket.objects.filter(Prefix = f'bus_full_day_errors_v2/{date_str}.csv')
    for obj in data_objects:
        public_bucket.copy({'Bucket': private_bucket.name, 'Key': obj.key}, obj.key)
    for obj in error_objects: 
        public_bucket.copy({'Bucket': private_bucket.name, 'Key': obj.key}, obj.key)